<a href="https://colab.research.google.com/github/hannari-python/tutorial/blob/master/trade_balance/trade_balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 貿易収支データの読み込み、前処理、可視化




## 読み込み
- 貿易収支データはkaggleのzanjibarさんのものを利用（毎月更新されています)
   - https://www.kaggle.com/zanjibar/japan-trade-statistics
- ライセンスは https://creativecommons.org/licenses/by-sa/4.0/
- ライセンスから再配布が可能であるため、今回はkaggleのライブラリをインストールしなくてもデータが利用できるよう、dropboxに利用データを格納しました。
   - https://www.dropbox.com/s/xsmjypizhhastyr/y_1997.db?dl=1
   - https://www.dropbox.com/s/ej8pj7tflk11cuu/trade_meta_data.db?dl=1
   - https://www.dropbox.com/s/jp03pp2gr14bmk0/codes.db?dl=1

- 最新データを利用したい場合は、長尾さんのリポジトリを参照ください


`!`の後にLinuxコマンドを書くとそのセルの命令はPythonのプログラムではなくLinuxのコマンドとして実行されます。

- `wget` はデータをダウンロードするのに使うことができるLinuxのコマンドです。
- `mv` はファイルをリネームをするのに使うことができるLinuxのコマンドです。

In [1]:
!wget https://www.dropbox.com/s/xsmjypizhhastyr/y_1997.db?dl=1
!wget https://www.dropbox.com/s/ej8pj7tflk11cuu/trade_meta_data.db?dl=1
!wget https://www.dropbox.com/s/jp03pp2gr14bmk0/codes.db?dl=1
!mv y_1997.db?dl=1 y_1997.db
!mv trade_meta_data.db?dl=1 trade_meta_data.db
!mv codes.db?dl=1 codes.db

--2020-08-22 10:00:16--  https://www.dropbox.com/s/xsmjypizhhastyr/y_1997.db?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/xsmjypizhhastyr/y_1997.db [following]
--2020-08-22 10:00:16--  https://www.dropbox.com/s/dl/xsmjypizhhastyr/y_1997.db
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc09fe8ddb06c08d736e0096c4ca.dl.dropboxusercontent.com/cd/0/get/A96cUyM1wVYGQaxrrnjX8mYG614mtbSnP3sGUwMJ5TaMGdBxQDyRucw_SEXOJsPCjc1O0sSPhGS6gwGRvhHtvQaStT4FRTIwhxvkDo-vVPHgPg/file?dl=1# [following]
--2020-08-22 10:00:16--  https://uc09fe8ddb06c08d736e0096c4ca.dl.dropboxusercontent.com/cd/0/get/A96cUyM1wVYGQaxrrnjX8mYG614mtbSnP3sGUwMJ5TaMGdBxQDyRucw_SEXOJsPCjc1O0sSPhGS6gwGRvhHtvQaStT4FRTIwhxvkDo-vVPHgPg/file?dl=1
Resolving uc09fe8ddb0

## ダウンロードの確認

- ダウンロードが終了したら、ファイルを確認してください
- 3つのファイルがダウンロードされたことが確認できます

## データの読み込み
- 次にダウンロードしたデータを読み込みます
- データベースの読み込みには、sqlite3とpandasを用います
- まずはDBにあるテーブルを調べます

In [97]:
import sqlite3 
import pandas as pd
import numpy as np

In [4]:
# まずは含まれるテーブルを参照する
conn = sqlite3.connect('/content/y_1997.db')
cursor = conn.cursor()
cursor.execute('select * from sqlite_master where type="table"') #sql文
print(cursor.fetchall())
conn.close()

[('table', 'year_from_1997', 'year_from_1997', 2, 'CREATE TABLE "year_from_1997" (\n"index" INTEGER,\n  "exp_imp" INTEGER,\n  "Year" INTEGER,\n  "Country" TEXT,\n  "Unit1" TEXT,\n  "Unit2" TEXT,\n  "Q1" INTEGER,\n  "Q2" INTEGER,\n  "Value" INTEGER,\n  "hs2" TEXT,\n  "hs4" TEXT,\n  "hs6" TEXT,\n  "hs9" TEXT\n)')]


# 関数にまとめてテーブルの中身を見る
- データベースのテーブル名などの表示
- pandasでデータを読む

In [63]:
def show_tables(db_path):
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()
  cursor.execute('select * from sqlite_master where type="table"') 
  table_name_list = []
  for i in cursor.fetchall():
    #print(i)
    table_name_list.append(i[1])
  conn.close()
  return table_name_list

In [64]:
trade_meta_table = show_tables('/content/trade_meta_data.db')
trade_meta_table

['country_jpn',
 'hs2_jpn',
 'hs2_eng',
 'hs4_eng',
 'hs6_jpn',
 'hs6_eng',
 'hs9_jpn',
 'hs9_eng',
 'country_eng',
 'custom',
 'hs4_jpn']

In [65]:
codes_table = show_tables('/content/codes.db')

In [66]:
codes_table

['country_jpn',
 'hs2_jpn',
 'hs2_eng',
 'hs4_eng',
 'hs6_jpn',
 'hs6_eng',
 'hs9_jpn',
 'hs9_eng',
 'country_eng',
 'hs4_jpn',
 'custom_tmp',
 'custom']

In [67]:
conn = sqlite3.connect('/content/codes.db')
sql = f'select * from {codes_table[0]}'
code_f = pd.read_sql(sql, conn)

In [68]:
code_f

,Country,Country_name,Area
0,103,大韓民国,Asia
1,104,北朝鮮,Asia
2,105,中華人民共和国,Asia
3,106,台湾,Asia
4,107,モンゴル,Asia
...,...,...,...
227,627,北マリアナ諸島(米),Oceania
228,628,パラオ,Oceania
229,701,指図式,Special_Area
230,702,不明,Special_Area


In [69]:
def get_table(db_path, table_name):
  conn = sqlite3.connect(db_path)
  sql = f'select * from {table_name}'
  df = pd.read_sql(sql, conn)
  return df

In [92]:
test_table = get_table('/content/trade_meta_data.db', codes_table[0])

In [94]:
test_table[:40]

,Country,Country_name,Area
0,103,大韓民国,Asia
1,104,北朝鮮,Asia
2,105,中華人民共和国,Asia
3,106,台湾,Asia
4,107,モンゴル,Asia
5,108,香港,Asia
6,110,ベトナム,Asia
7,111,タイ,Asia
8,112,シンガポール,Asia
9,113,マレーシア,Asia


## 課題1
- dbの持つテーブルの中身をread_sql関数を使って読んでみましょう

# 各国の年間の日本に対する輸出入のデータを取得する
- y_1997.dbのyear_from_1997を読み込みましょう
- その際、全て読み込むとサイズが大きくなるので、金額を合計して読み込みます

In [75]:
conn = sqlite3.connect('/content/y_1997.db')
sql = 'select Country, exp_imp, Year, sum(Value) as Value from year_from_1997 group by Country, Year, exp_imp'
data = pd.read_sql(sql, conn)
conn.close()

In [77]:
data #exp_impは１が輸出、2が輸入

,Country,exp_imp,Year,Value
0,103,1,1997,3153238334
1,103,2,1997,1762757028
2,103,1,1998,2004542479
3,103,2,1998,1577249772
4,103,1,1999,2606234190
...,...,...,...,...
10217,702,2,2016,183523
10218,702,2,2017,330381
10219,702,2,2018,490757
10220,702,2,2019,247949


## 課題2
- 国別のデータではなく、各年の輸出入の合計のデータを作成してみましょう

# exp_impが数字では分かり難い！！！
- ちなみに1が輸出、2が輸入
- 辞書を作ってmapで割り当てましょう

In [122]:
exp_imp_dict = {1: '輸出', 2:'輸入'}

data['輸出入'] = data['exp_imp'].map(lambda x: exp_imp_dict[x])

In [123]:
data

,Country,exp_imp,Year,Value,country_name,輸出入
0,103,1,1997,3153238334,大韓民国,輸出
1,103,2,1997,1762757028,大韓民国,輸入
2,103,1,1998,2004542479,大韓民国,輸出
3,103,2,1998,1577249772,大韓民国,輸入
4,103,1,1999,2606234190,大韓民国,輸出
...,...,...,...,...,...,...
10217,702,2,2016,183523,不明,輸入
10218,702,2,2017,330381,不明,輸入
10219,702,2,2018,490757,不明,輸入
10220,702,2,2019,247949,不明,輸入


## 国名データを使いたい！！！
- 上と同じで良いんでしょ!
- そのため、番号と国名を辞書にしてそれを使って国名の列をを作る

In [124]:
# インデックスに国番号、valueに国名を持つSeriesを作成し、to_dictメソッドを使って辞書を作成する
code_f
country_code = code_f[["Country", "Country_name"]]
country_code = country_code.set_index("Country")
country_dict = country_code.to_dict()["Country_name"]

In [125]:
# mapメソッドとlambda式と辞書を用いて国名列を作成する・・キーエラー・・・
# 148がない・・・
data["country_name"] = data["Country"].map(lambda x: country_dict[x])

KeyError: ignored

In [126]:
# KeyErrorを乗り越える関数を作成する（

def select_country(x):
  n = 0
  try:
    return country_dict[x]
  except:
    n += 1
    return f'不明_{n}'

In [127]:
# この関数をlambda式に渡す
data["country_name"] = data["Country"].map(lambda x: select_country(x))

In [128]:
data.country_name.unique()

array(['大韓民国', '北朝鮮', '中華人民共和国', '台湾', 'モンゴル', '香港', 'ベトナム', 'タイ',
       'シンガポール', 'マレーシア', 'ブルネイ', 'フィリピン', 'インドネシア', 'カンボジア', 'ラオス',
       'ミャンマー', 'インド', 'パキスタン', 'スリランカ', 'モルディブ', 'バングラデシュ', '東ティモール',
       'マカオ', 'アフガニスタン', 'ネパール', 'ブータン', 'イラン', 'イラク', 'バーレーン', 'サウジアラビア',
       'クウェート', 'カタール', 'オマーン', 'イスラエル', 'ヨルダン', 'シリア', 'レバノン',
       'アラブ首長国連邦', '不明_1', 'イエメン', 'アゼルバイジャン', 'アルメニア', 'ウズベキスタン',
       'カザフスタン', 'キルギス', 'タジキスタン', 'トルクメニスタン', 'ジョージア', 'ヨルダン川西岸及びガザ',
       'アイスランド', 'ノルウェー', 'スウェーデン', 'デンマーク', '英国', 'アイルランド', 'オランダ',
       'ベルギー', 'ルクセンブルク', 'フランス', 'モナコ', 'アンドラ', 'ドイツ', 'スイス', 'アゾレス(葡)',
       'ポルトガル', 'スペイン', 'ジブラルタル(英)', 'イタリア', 'マルタ', 'フィンランド', 'ポーランド',
       'ロシア', 'オーストリア', 'ハンガリー', 'セルビア', 'アルバニア', 'ギリシャ', 'ルーマニア',
       'ブルガリア', 'キプロス', 'トルコ', 'エストニア', 'ラトビア', 'リトアニア', 'ウクライナ', 'ベラルーシ',
       'モルドバ', 'クロアチア', 'スロベニア', 'ボスニア・ヘルツェゴビナ', 'マケドニア旧ユーゴスラビア共和国',
       'チェコ', 'スロバキア', 'モンテネグロ', 'コソボ', 'グリーンランド(デンマーク)', 'カナダ',
       'サンピエール及びミクロン(仏)', 

In [129]:
data

,Country,exp_imp,Year,Value,country_name,輸出入
0,103,1,1997,3153238334,大韓民国,輸出
1,103,2,1997,1762757028,大韓民国,輸入
2,103,1,1998,2004542479,大韓民国,輸出
3,103,2,1998,1577249772,大韓民国,輸入
4,103,1,1999,2606234190,大韓民国,輸出
...,...,...,...,...,...,...
10217,702,2,2016,183523,不明,輸入
10218,702,2,2017,330381,不明,輸入
10219,702,2,2018,490757,不明,輸入
10220,702,2,2019,247949,不明,輸入


# Plotlyを使ってデータを可視化する
- Plotlyはグラフを描画するためのPythonパッケージです
   - https://plotly.com/python/
- Colabにプリインストールされているものの、バージョンが古いため、pip install --upgrade plotlyで更新してから使います[初めに行っています]
- まずはPlotlyExpressという、plotlyのラッパ（簡単に使えるモジュール）を使って可視化します
- PlotlyExpressの使い方としては関数名をグラフの種類の選択したあと、データフレームを渡し、そのあとの引数に使いたい列名を渡します
  - https://plotly.com/python/plotly-express/


In [137]:
import plotly.express as px

In [142]:
# 1か国の描画
data_korea = data[data['country_name'] == '大韓民国']
px.line(data_korea, x='Year', y='Value', color='輸出入', title='大韓民国')

## 課題3
- PlotlyExpressのサイトを見ながら、何か他のグラフを作成してみてください
    - https://plotly.com/python/plotly-express/

# Plotly Graph Objectsを使う
- PlotlyExpressで十分なことも多いが、やりたいことができない場合もある
- このデータの形の場合、複数国のデータを可視化する（輸出入）は難しい


In [147]:
# 複数国の描画(うまく描画できない・・・)
two_countries = ['大韓民国', '中華人民共和国']
data_two = data[data['country_name'].isin(two_countries)]
px.line(data_two, x='Year', y='Value', color='輸出入', title='大韓民国と中華人民共和国')

- ラッパはできることに限界があるので、plotly graph objectsを使ってみます。


In [152]:
import plotly.graph_objects as go  
from plotly.subplots import make_subplots 

In [153]:
two_countries = ['大韓民国', '中華人民共和国']
data_two = data[data['country_name'].isin(two_countries)]
fig = go.Figure()

for cnt in two_countries:
  data_one = data_two[data_two['country_name'] == cnt]
  data_one_exp = data_one[data_one['exp_imp'] == 1]
  data_one_imp = data_one[data_one['exp_imp'] == 2]

  fig.add_trace(go.Scatter(x=data_one_exp['Year'], y=data_one_exp['Value'], name=f'{cnt} 輸出'))
  fig.add_trace(go.Scatter(x=data_one_imp['Year'], y=data_one_imp['Value'], name=f'{cnt} 輸入'))

fig.show()

In [161]:
# 貿易収支も描画

two_countries = ['大韓民国', '中華人民共和国']
data_two = data[data['country_name'].isin(two_countries)]

fig = make_subplots(rows=2, cols=1)

for cnt in two_countries:
  data_one = data_two[data_two['country_name'] == cnt]
  data_one_exp = data_one[data_one['exp_imp'] == 1].reset_index(drop=True)
  data_one_imp = data_one[data_one['exp_imp'] == 2].reset_index(drop=True)
  data_one_balance = data_one_exp['Value'] - data_one_imp['Value'] 
  data_one_balance = pd.concat([data_one_exp['Year'], data_one_balance], axis=1)

  fig.add_trace(go.Scatter(x=data_one_exp['Year'], y=data_one_exp['Value'], name=f'{cnt} 輸出'), row=1, col=1)
  fig.add_trace(go.Scatter(x=data_one_imp['Year'], y=data_one_imp['Value'], name=f'{cnt} 輸入'), row=1, col=1)
  fig.add_trace(go.Bar(x=data_one_balance['Year'], y=data_one_balance['Value'], name=f'{cnt} 収支'), row=2, col=1)

fig.show()

In [162]:
# あとはDashのアプリケーションの作成